# GOAL
The goal of this notebook is to first clean up two location databases, one of all addresses in Oakland, and one of vulnerable buildings. We also use similarity search to match similar street names and cluster similar statuses of buidlings.
Secondly, we download the image data to individual folders to enable learning later.

### Clean up mixed types of house numbers

In [41]:
import pandas as pd
def ignore_non_int(df, column = 'NUMBER'):
    """Returns a df with all rows removed that 
    do not have integers in 'column'
    
    Parameters
    ----------
    df : pd.Dataframe 
        The input dataframe that contains
        'column'
    column : str, optional
        The name of the column that should
        only contain integers

    Returns
    -------
    pd.DataFrame
        The dataframe with rows removed
    """
    def to_int(x):
        try:
            return int(x)
        except:
            return x
    df[column]=df[column].apply(lambda x: to_int(x))
    df['dtypes']=df[column].apply(lambda x: type(x))
    df=df.loc[df['dtypes']==type(1)]
    df.drop('dtypes',axis=1,inplace=True)
    df[column]=df[column].apply(lambda x: to_int(x))
    return df
df_openaddr = pd.read_csv('Locations/alameda.csv')
df_openaddr = ignore_non_int(df_openaddr)
df_openaddr['NUMBER'].dtypes

dtype('int64')

### Load Oakland list of vulnerable buildings and do some pre_processings

In [42]:
def seperate_house_numbers(df):
    """Returns a df that splits the 'address'
    column in house number, street number and 
    building
    
    Parameters
    ----------
    df : pd.Dataframe 
        The input dataframe that contains an
        'address' column

    Returns
    -------
    pd.DataFrame
        The dataframe with seperate columns
        for NUMBER, STREET, BUILDING
    """
    #Seperate Street and number
    df[['NUMBER','STREET']] = pd.DataFrame(df['address'].str.split(' ',1).tolist(),
                                       columns = ['NUMBER','STREET'])
    #Seperate Street and building
    df[['STREET','BUILDING']]=pd.DataFrame(df['STREET'].str.split(' - ',1).tolist(),
                                       columns = ['STREET','BUILDING'])
    return df

df_vul = seperate_house_numbers(pd.read_csv('Locations/Oakland.csv'))
df_vul.head()

,parcel_number,address,status_long,latitude,longitude,status_short,NUMBER,STREET,BUILDING
0,022 030500100,2232 IVY DR,Ask for exemption - did not review,37.800755,-122.245017,Exempt,2232,IVY DR,None
1,027 085401703,2858 BROOKDALE AVE,Incomplete Evaluation,37.793341,-122.217329,Incomplete Evaluation,2858,BROOKDALE AVE,None
2,022 030901400,323 HANOVER AVE,Incomplete Evaluation,37.803079,-122.251108,Incomplete Evaluation,323,HANOVER AVE,None
3,010 077102000,246 GRAND AVE,Level 2 - Missing slope,37.809921,-122.258968,Incomplete Evaluation,246,GRAND AVE,None
4,039 330305500,6862 ARTHUR ST,Exempt - less than 5 units,37.765986,-122.182236,Exempt,6862,ARTHUR ST,None


### Find housenumbers of buildings which were combined, i.e. Telegraph Ave 5678+5683

In [43]:
# Split the odd numbers into seperate columns
def tidy_split(df, column, sep='|', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.
    
    https://github.com/cognoma/genes/blob/721204091a96e55de6dcad165d6d8265e67e2a48/2.process.py

    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df


df_vul=tidy_split(df_vul,'NUMBER',sep='+')
df_vul=ignore_non_int(df_vul)
df_vul['NUMBER'].dtypes

dtype('int64')

### Match street names in df_vul with those in df_openaddr via similarity search

In [44]:
import difflib

def align_names(df_1,df_2,column_name = 'STREET'):
    """Renames all elements in column 'colum_name' of 
    df_1 to the closest match in  column 'colum_name' 
    of df_2.
    
    Parameters
    ----------
    df_1 : pd.Dataframe 
        The dataframe whose column 'column_name'
        should be renamed.
    df_2 : pd.Dataframe 
        The input dataframe whose terminology of 
        column 'column_name' should be applied to 
        df_1.
    column_name: str, optional
        The name of the column (must be part of both
        dateframes) whose terminology should be
        aligned.

    Returns
    -------
    pd.DataFrame
        df_1 with the column 'column_name' changed to
        the closest names of the column 'column_name'
        in df_2.
    """
    

    # Find all unique elements in column column_name in df_1 and df_2
    unique_streets1 = df_1[column_name].unique()
    unique_streets2 = df_2[column_name].unique()

    # Find most similar entried in column 'column_name' in df_2
    matching_streets = []
    for unique_street1 in unique_streets1:
        matching_streets.append(difflib.get_close_matches(unique_street1, unique_streets2)[0])

    #Create a dictionary that can translate names
    df1_to_df2 = dict(zip(unique_streets,matching_streets))

    # Rename all names column 'column_name' in df_1 to the notation of df_2
    df_1[column_name]=df_1[column_name].apply(lambda x: df1_to_df2[x])
    
    return df_1

df_openaddr_oakland = df_openaddr[df_openaddr['CITY']=='OAKLAND']
df_vul_matched = align_addresses(df_vul,df_openaddr_oakland)
df_vul_matched.head()

,parcel_number,address,status_long,latitude,longitude,status_short,NUMBER,STREET,BUILDING
0,022 030500100,2232 IVY DR,Ask for exemption - did not review,37.800755,-122.245017,Exempt,2232,IVY DR,None
1,027 085401703,2858 BROOKDALE AVE,Incomplete Evaluation,37.793341,-122.217329,Incomplete Evaluation,2858,BROOKDALE AV,None
2,022 030901400,323 HANOVER AVE,Incomplete Evaluation,37.803079,-122.251108,Incomplete Evaluation,323,HANOVER AV,None
3,010 077102000,246 GRAND AVE,Level 2 - Missing slope,37.809921,-122.258968,Incomplete Evaluation,246,GRAND AV,None
4,039 330305500,6862 ARTHUR ST,Exempt - less than 5 units,37.765986,-122.182236,Exempt,6862,ARTHUR ST,None


# Here we are doing some consistency check of our data

### Make sure most addresses from df_vul_matched are contained in df_openaddr

In [24]:
def address_not_contained(df_1,df_2):
    missing = []
    for index,row in df_1.iterrows():
        if not (((df_2['STREET'] == row['STREET'])
                & (df_2['NUMBER'] == row['NUMBER'])).any()):
                missing.append(index)
    return len(missing)
address_not_contained(df_vul_matched,df_openaddr_oakland)

33

Just 33 are missing - that is ok for now. Next we find all streets that have at least 5 vulnerable buildings and find all other buildings in those streets. 

In [29]:
def buildings_in_popular_street(df1,df2,threshold = 5):
    """Returns those addresses in df2 which are in streets
    in df1 that contain more than 'threshold' houses.
    
    Parameters
    ----------
    df1 : pd.Dataframe 
        List of addresses with houses for which the number
        of buildings per street is determined. 
     df2 : pd.Dataframe 
        List of addresses with houses that should be filtered.
        This df should contain at least some of the same street
        names as df1. 
    threshold: int, optional
        The threshold for the number of houses per street in df1,
        that should be used to filter df2

    Returns
    -------
    pd.DataFrame
        df2 that contains only those buildings that are in 
        popular streets (more than threshold buildings) in df1
    """

    counts=pd.DataFrame(df1['STREET'].value_counts(dropna=False))
    popular_street = counts[counts>=threshold].dropna().index.tolist()
    return df2[df2['STREET'].isin(popular_street)]

df_openaddr_oakland_vul_streets = buildings_in_popular_street(df_vul_matched,df_openaddr_oakland)
df_openaddr_oakland_vul_streets.head(2)

,Unnamed: 0,LON,LAT,NUMBER,STREET,UNIT,CITY,DISTRICT,REGION,POSTCODE,ID,HASH,Numberlen
0,0,-122.261672,37.837729,5132,TELEGRAPH AV,NaN,OAKLAND,NaN,NaN,94609,14-1226-15,2b13400a4d483c21,4
1,1,-122.261910,37.837433,5110,TELEGRAPH AV,NaN,OAKLAND,NaN,NaN,94609,14-1226-15,fe2c0215b85bd442,4


### Make a function that returns only buildings that are contained in the first, second or in both dataframes

In [31]:
def in_which_df(df1,df2,columns = ['STREET','NUMBER'], output = 'first'):
    """Returns a df with rows only contained in df1 (first), 
    in both (both), or only in df2 (right) by comparing 'columns'.
    
    Parameters
    ----------
    df1 : pd.Dataframe 
        The first input dataframe 
    df2 : pd.Dataframe 
        The second input dataframe 
    columns : list of str, optional
        The name of the columns where
        df1 and df2 should coincide.
    output: str, optional
        'first': returns rows that are only in df1
        'both': returns rows that are both in df1 and df2
        'second': returns rows that are only in df2

    Returns
    -------
    pd.DataFrame of rows that are only in 'output'
        
    """    
    df=df1.drop_duplicates().merge(df2.drop_duplicates(),on=columns,
                                      how='outer', indicator=True)
    if output == 'first':
        return df[df['_merge']=='left_only']
    elif output == 'second':
        return df[df['_merge']=='right_only']
    elif output == 'both':
        return df[df['_merge']=='both']
    else:
        return pd.DataFrame()

### Make a list of non-vulnerable buildings from streets that contain more than 5 vulnerable buildings

This will give us the class '0' (negative) dataset of buildings.

In [34]:
non_vul_df = in_which_df(df_openaddr_oakland_vul_streets,df_vul_matched,output = 'first')
non_vul_df.head(3)

,Unnamed: 0,LON,LAT,NUMBER,STREET,UNIT,CITY,DISTRICT,REGION,POSTCODE,...,HASH,Numberlen,parcel_number,address,status_long,latitude,longitude,status_short,BUILDING,_merge
0,0.0,-122.261672,37.837729,5132,TELEGRAPH AV,NaN,OAKLAND,NaN,NaN,94609.0,...,2b13400a4d483c21,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,1.0,-122.261910,37.837433,5110,TELEGRAPH AV,NaN,OAKLAND,NaN,NaN,94609.0,...,fe2c0215b85bd442,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,3.0,-122.261959,37.837303,5100,TELEGRAPH AV,NaN,OAKLAND,NaN,NaN,94609.0,...,5be8273266922da0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


### Do some final clean-up of vulnerable buildings and cluster them by their statuses

In [38]:
def find_similar_clusters(df, column_name,cutoff=.8):
    """Returns a clusters of similar terms in column_name of the
    DataFrame df. The cut_off variable determines the threshold
    for similarity
    
    Parameters
    ----------
    df : pd.Dataframe 
        The input dataframe with a column 'column_name'
    column_name : str
        The name of the columns of categories to be clustered
    cutoff : float, optional
        Value betweeen 0 and 1 that gives the similarity threshol

    Returns
    -------
    list of list of expressions that are similar.
        
    """   
    #     Make a list of all unique items in column
    cats = list(pd.DataFrame(df[column_name].value_counts()).index)

    # Find similar clusters
    logical_sets = []
    for cat in cats:
        cluster = difflib.get_close_matches(cat, cats, n=10, cutoff=0.8)
        cluster.sort()
        logical_sets.append(cluster)

        # Delete duplicates
    unique_list = []
    for logical_set in logical_sets:
        if not logical_set in unique_list:
            unique_list.append(logical_set)
    return unique_list
df_vul_clusters = find_similar_clusters(df_vul_matched, column_name ='status_long')
df_vul_clusters[:2]

[['Done - Level 1', 'Done - Level 1 '],
 ['Incomplete Evaluation', 'Incomplete Evaluation - no data']]

## Download Street View Images for Datasets via Google API

In [76]:
from os import path, makedirs
import time
import json
import requests
def download_images(df,api_key='', dir_path ='images',class_name = 'vulnerable', 
                    title = 'view', category = '',pitch=10):
    """Downloads all Street view for all rows with addresses in df. 
    Parameters
    ----------
    df : pd.Dataframe 
        The df with rows that contain at least the columns
        'CITY', 'STREET', 'NUMBER', 'LAT', 'LON'
    api_key : str, optional
        Google Maps Street View API key, without the key only 
        the request url is returned
    dir_path : str, optional
        The path of the download folder
    class_name : str, optional
        The label of the data to download
    title : str, optional
        The name of the image file
    category : str, optional
        An additional layer of nesting within
        the class
    pitch : int, optional
        the camera angle of the street view image,
        0 is horizontal
    output: str, optional
        'first': returns rows that are only in df1
        'both': returns rows that are both in df1 and df2
        'second': returns rows that are only in df2

    Returns
    -------
    Downloads all images to their respective folders as jpg 
    and saves all other info in a json file with the same name.
    returns True once completed. 
        
    """
    
    def empty_str(x):
        if str(x)=='nan':
            return ''
        else:
            return x
    total_path = ('/').join([dir_path,class_name])
    if not path.isdir(total_path):
        makedirs(total_path)
    if category:
            category = category.replace(' ','-').replace('/','-')
            total_path=('/').join([total_path,category])
            if not path.isdir(total_path):
                makedirs(total_path)
        
        
    for index,row in df.iterrows():
        address_str = ('+').join([str(row['NUMBER']),
                                        row['STREET'],
                                        row['CITY'],
                                            ]).replace(' ','+')
        try:
            unit = empty_str(row['UNIT'])
            address_str+='+'+unit
        except:
            pass
        try:
            postcode = str(row['POSTCODE'])
            address_str+='+'+postcode
        except:
            pass                                 
        file_name = ('_').join([title,
                                class_name,
                                str(int(time.time())),
                                "{:+f}".format(row['LAT']),
                                "{:+f}".format(row['LON']),
                                str(pitch),
                                address_str
                               ]) 
        if category:
            file_name+='+'+category

        url='https://maps.googleapis.com/maps/api/streetview?source=outdoor&size=640x640'
        url+= '&pitch='+str(pitch)
        url+= '&key='+api_key
        url+= '&location='+address_str
        if not api_key:
            return url
        response = requests.get(url)
        file_path = path.join(total_path,file_name)
        if response.status_code == 200:
            with open(file_path+".jpg", 'wb') as f:
                f.write(response.content)
            with open(file_path+".json", 'w') as f:
                f.write(str(row.to_json()))
    return True

## Download Non-vulnerable buildings

In [16]:
download_images(non_vul_df,class_name='non_vulnerable')

## Download vulnerable buildings

### Iterate through the diffent categorical clusters of df_vul_matched and download images into separate folders

In [78]:
# First rename some columns and set city name
df_vul_matched=df_vul_matched.rename(index=str, columns={"latitude": "LAT", "longitude": "LON"})
df_vul_matched['CITY']='OAKLAND'

for cluster in df_vul_clusters:
    print('current category', item[0])
    download_images(df_vul_matched[df_vul_matched['status_long'].isin(cluster)],
                           class_name='vulnerable',
                           category = item[0],test=False)

current category Done - Level 1
current category Incomplete Evaluation
current category Incomplete Evaluation - Unclaimed
current category Done - Level 2 Required
current category Exempt - engineer's letter
current category Exempt - no large openings
current category Level 1 - missing data
current category Level 2 - Missing slope
current category Exempt - less than 5 units
current category Exempt - retrofitted
current category Ask for exemption - did not review
current category Exempt - city inspection
current category Done - Level 2 in process
current category Exempt - no parking/commercial
current category Incomplete Evaluation
current category Done - Level 2 in process - requires retrofit
current category Exempt - Garage In basement
current category Exempt - built after 1990


# All done - we got our positive and negative training dataset